# Import packages

In [26]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import string
import math
from nltk.stem import PorterStemmer
import warnings
warnings.filterwarnings("ignore")

# Load dataset

In [3]:
dataset = {
    'doc_id': [],
    'doc_text': []
}
num_of_docs = 1095
for i in tqdm(range(1, num_of_docs+1)):
    with open(f'./data/{i}.txt', mode='r') as f:
        doc_text: str = f.read()
    
    dataset['doc_id'].append(i)
    dataset['doc_text'].append(doc_text)

dataset: pd.DataFrame = pd.DataFrame(dataset, index=dataset['doc_id'])
dataset.head()

100%|██████████| 1095/1095 [00:00<00:00, 3260.16it/s]


,doc_id,doc_text
1,1,the white house is also keeping a close watch ...
2,2,"turning to news overseas, a tense political sh..."
3,3,Pressing a strategy of legal challenges and po...
4,4,"In Yugoslavia, the Democratic opposition will ..."
5,5,The Yugoslavia opposition is urging its suppor...


# Text preprocessing

In [4]:
with open('./stopwords.txt', mode='r') as f:
    stopwords = f.read().split()

def remove_newline_symbol(text: str) -> str:
    return text.replace('\n', '')

def remove_punctuation(text: str) -> str:
    punctuation = string.punctuation + '-'

    return ''.join([char for char in text if char not in punctuation])

def remove_digital(text: str) -> str:
    return ''.join([char for char in text if not char.isdigit()])

def remove_stopwords(text: str) -> str:
    return ' '.join([word for word in text.split() if word not in stopwords])

def lowercase(text: str) -> str:
    return text.lower()

def stemming(text: str) -> str:
    stemmer = PorterStemmer()
    return ' '.join([stemmer.stem(word) for word in text.split()])

def tokenize(text: str) -> list[str]:
    return text.split()

def text_preprocessing_pipeline(text: str) -> list[str]:
    text = remove_newline_symbol(text)
    text = remove_punctuation(text)
    text = remove_digital(text)
    text = lowercase(text)
    text = remove_stopwords(text)
    text = stemming(text)

    return text

In [5]:
preprocessed_datasets = dataset.copy()
preprocessed_datasets['doc_text'] = preprocessed_datasets['doc_text'].progress_apply(text_preprocessing_pipeline)
preprocessed_datasets.head()

100%|██████████| 1095/1095 [00:10<00:00, 107.05it/s]


,doc_id,doc_text
1,1,white hous keep close watch yugoslavia opposit...
2,2,news oversea tens polit showdown yugoslavia su...
3,3,press strategi legal challeng popular pressur ...
4,4,yugoslavia democrat opposit nationwid campaign...
5,5,yugoslavia opposit urg support civil disobedi ...


# Get dictionary

In [6]:
def get_dictionary(dataset: pd.DataFrame) -> list[str]:
    dictionary = []
    for doc_text in tqdm(dataset['doc_text']):
        dictionary.extend(doc_text.split())
    
    return list(set(dictionary))

dictionary = get_dictionary(preprocessed_datasets)
print(dictionary[:10])

100%|██████████| 1095/1095 [00:00<00:00, 31483.80it/s]

['lag', 'multifacet', 'buraika', 'jingqian', 'wouldv', 'earn', 'slice', 'spring', 'coverup', 'complianc']


# Get document frequency (df)

In [7]:
def compute_df_and_idf(dataset: pd.DataFrame, dictionary: list[str]) -> pd.DataFrame:
    term2df_and_idf: pd.DataFrame = pd.DataFrame(dictionary, columns=['term'])

    term2df_and_idf['df'] = term2df_and_idf['term'].progress_apply(
        lambda term: dataset['doc_text'].str.contains(term).sum()
    )

    term2df_and_idf['idf'] = np.log(len(dataset) / term2df_and_idf['df'])

    term2df_and_idf.set_index('term', inplace=True)

    return term2df_and_idf

term2df_and_idf = compute_df_and_idf(preprocessed_datasets, dictionary)
term2df_and_idf.head()

100%|██████████| 13189/13189 [00:29<00:00, 440.62it/s]


,df,idf
term,,
lag,101,2.383389
multifacet,2,6.305362
buraika,1,6.998510
jingqian,1,6.998510
wouldv,1,6.998510


# Get normalized tf-idf vector

In [8]:
def get_tf_idf_vector(dataset: pd.DataFrame, term2df_and_idf: pd.DataFrame) -> dict[int, np.ndarray]:
    doc2tf_idf_vector = dict()

    for i in tqdm(range(1, len(dataset)+1)):
        doc_text = dataset.loc[i, 'doc_text']

        tf = pd.Series(doc_text.split()).value_counts().to_dict()
        tf_vector: pd.Series = term2df_and_idf.index.map(tf).fillna(0)
        tf_idf_vector = (tf_vector * term2df_and_idf['idf']) / np.linalg.norm(tf_vector * term2df_and_idf['idf'])
       
        doc2tf_idf_vector[i] = tf_idf_vector.values

    return doc2tf_idf_vector

doc2tf_idf_vector: dict[int, np.ndarray] = get_tf_idf_vector(preprocessed_datasets, term2df_and_idf)

100%|██████████| 1095/1095 [00:03<00:00, 334.05it/s]


# Hierarchical Agglomerative Clustering

## Computing cosine similarity

In [9]:
def cosine_similarity(doc1: np.ndarray, doc2: np.ndarray) -> float:
    return np.dot(doc1, doc2) # doc 已經 normalize 過，所以不用再除以兩個 doc 向量的長度

## Computing cluster similarity (complete link)

In [10]:
def complete_link(cluster_j: int, cluster_i: int, cluster_m: int, C: np.ndarray) -> float:
    # 比較 i 與 j 的相似度，和新加入的 m 與 j 的相似度，取小的當作新的相似度
    if C[cluster_i, cluster_j] < C[cluster_m, cluster_j]:
        return C[cluster_i, cluster_j]
    else:
        return C[cluster_m, cluster_j]

## Heap

In [45]:
class Heap:
    def __init__(self):
        self.data: list[tuple[float, int, int]] = [(-1, -1, -1)] # 0 號位置不放東西，方便之後的 heap 運算
        self.n: int = 0 # 目前 heap 的大小
    
    def insert_data(self, x: tuple[float, int, int]): # tuple: [相似度, cluster_i, cluster_j]
        self.n += 1
        self.data.append(tuple())
        idx = self.n

        while idx > 1 and x[0] > self.data[math.floor(idx/2)][0]:
            self.data[idx] = self.data[math.floor(idx/2)]
            idx = math.floor(idx/2)
        
        self.data[idx] = x

    def pop_data(self) -> tuple[float, int, int]:
        x = self.data[1]
        self.data[1] = self.data[self.n]
        self.n -= 1

        idx = 1
        while idx <= (self.n/2):
            if self.data[idx*2][0] > self.data[idx*2+1][0]:
                child_idx = idx*2
            else:
                child_idx = idx*2+1

            if self.data[idx][0] > self.data[child_idx][0]:
                break

            self.data[idx], self.data[child_idx] = self.data[child_idx], self.data[idx]
            idx = child_idx
        
        return x

## Computing HAC

In [11]:
def argmax(matrix: np.ndarray, I: np.ndarray) -> tuple[int, int]:
    max_value = -np.inf
    cluster_i = -1
    cluster_m = -1
    for i in range(len(matrix)):
        if I[i] == 0: # 如果 cluster i 已經被 merge 到其他 cluster，就不用再看了
            continue
        for j in range(i+1, len(matrix)):
            if I[j] == 0: # 如果 cluster j 已經被 merge 到其他 cluster，就不用再看了
                continue

            if matrix[i, j] > max_value:
                max_value = matrix[i, j]
                cluster_i = i
                cluster_m = j
    
    return cluster_i, cluster_m

In [ ]:
def efficent_HAC(doc2tf_idf_vector: dict[int, np.ndarray]):
    # C[i][j]: the similiarity between cluster i and cluster j
    C = np.ones((len(doc2tf_idf_vector), len(doc2tf_idf_vector)))  
    # I: indicate which clusters are still available to be merged
    I = np.ones(len(doc2tf_idf_vector))
    # A: log the merge history
    A = []

    heap = Heap()

    # Initialize
    for n in tqdm(range(len(doc2tf_idf_vector))):
        for i in range(n+1, len(doc2tf_idf_vector)):
            similarity = cosine_similarity(doc2tf_idf_vector[n+1], doc2tf_idf_vector[i+1])
            # 這是一個對稱矩陣
            C[n ,i] = similarity
            C[i, n] = similarity

            # Insert into heap
            if n != i:
                heap.insert_data((similarity, n, i))
        
        I[n] = 1
    
    # Merge: 進行 N-1 次 merge
    for _ in tqdm(range(len(doc2tf_idf_vector)-1)): # 這邊 -1，因為 complete binary tree 有 N 個 external nodes (docs) 代表有 N-1 個 internal nodes (merge)
        _, cluster_i, cluster_m = heap.pop_data()
        while I[cluster_i] == 0 or I[cluster_m] == 0: # 如果 cluster_i 或 cluster_m 已經被 merge 過了，就再 pop 一次
            _, cluster_i, cluster_m = heap.pop_data()
        A.append((cluster_i+1, cluster_m+1)) # cluster_i 和 cluster_m 是從 0 開始算的，所以要 +1

        # Update C: 因為 cluster_i merged 了 cluster_j，所以要重新計算新的 cluster 和其他 cluster 的相似度
        for j in range(len(doc2tf_idf_vector)):
            if j != cluster_i and j != cluster_m and I[j] == 1: # 不需要計算自己群跟自己群成員的相似度，也不需要計算已經被 merge 的群 (I[j] == 0)
                new_similarity = complete_link(j, cluster_i, cluster_m, C)
                C[cluster_i, j] = new_similarity
                C[j, cluster_i] = new_similarity

                heap.insert_data((new_similarity, cluster_i, j))

        I[cluster_m] = 0 # cluster_m 被併掉了

    return A
         

In [82]:
A = efficent_HAC(doc2tf_idf_vector)
print(A[:10])

100%|██████████| 1094/1094 [00:14<00:00, 74.35it/s] 

[(48, 49), (527, 529), (943, 944), (595, 596), (621, 622), (8, 9), (101, 106), (564, 565), (564, 595), (211, 212)]


# Output

In [83]:
K = [8, 13, 20] # number of clusters
def get_cluster(A: list[tuple[int, int]]):
    for k in tqdm(K):
        clusters = dict()
        for i in range(num_of_docs-k): # num_of_docs-k 次 merge 後，就會有 k 個 clusters
            cluster_i, cluster_m = A[i]
            clusters[cluster_i] = clusters.get(cluster_i, []) + clusters.get(cluster_m, []) + [cluster_m]
            if cluster_m in clusters:
                del clusters[cluster_m]

        # Write to file
        with open(f'./{k}.txt', mode='w') as f:
            for cluster in clusters:
                c = sorted([cluster] + clusters[cluster])
                for doc in c:
                    f.write(f'{doc}\n')
                f.write('\n')

In [80]:
get_cluster(A)

100%|██████████| 3/3 [00:00<00:00, 89.58it/s]
